In [1]:
import pandas as pd

In [36]:
df = pd.read_csv('/Users/kaitlinsinger/wa_notify/DEN_future/zotero_sql_querry/simplified_wide_df_with_date_loc.csv')

In [37]:
df.head()

,Unnamed: 0,source,pm_start_date,pm_end_date,pm_location,adoption_rate,cases_assigned_count,cases_interviewed_count,cases_interviewed_perc,cases_named_contacts_count,...,notifications_sent_count_weekly,notifications_sent_max,num_users_count,num_users_max,num_users_upper_bound,percent_naming_contacts,staff_hired_count,staff_hired_per_10kpop,user_cases_notifying,users_with_code
0,0,A Multifaceted Evaluation of a COVID-19 Contac...,04/01/2021,06/30/2020,King County,NaN,NaN,NaN,76.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A Multifaceted Evaluation of a COVID-19 Contac...,07/01/2020,06/30/2021,King County,NaN,52317.0,42900.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A Multifaceted Evaluation of a COVID-19 Contac...,07/01/2020,09/30/2020,King County,NaN,NaN,NaN,87.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A Patient-Initiated Digital COVID-19 Contact N...,05/18/2020,08/17/2020,United States*,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,A State Health Department and Health Informati...,06/15/2020,08/28/2021,Maryland,NaN,498739.0,352356.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
#[x for x in df.columns if x.endswith('count') or x.endswith('perc')]
df.columns

Index(['Unnamed: 0', 'source', 'pm_start_date', 'pm_end_date', 'pm_location',
       'adoption_rate', 'cases_assigned_count', 'cases_interviewed_count',
       'cases_interviewed_perc', 'cases_named_contacts_count',
       'cases_not_assigned_count', 'cases_positive_from_test_mean',
       'cases_positive_from_test_med', 'cases_reached_count',
       'cases_reached_from_symptom_mean', 'cases_reached_from_symptom_med',
       'cases_reached_from_test_max', 'cases_reached_from_test_mean',
       'cases_reached_from_test_med', 'cases_reached_perc',
       'contacts_assigned_count', 'contacts_infected_count',
       'contacts_infected_perc', 'contacts_interviewed_count',
       'contacts_interviewed_perc', 'contacts_named_cases_naming',
       'contacts_named_cases_naming_ratio',
       'contacts_named_cases_naming_ratio_mean', 'contacts_named_count',
       'contacts_named_count_household', 'contacts_named_perc',
       'contacts_named_perc_household', 'contacts_named_ratio',
       'cont

In [43]:
# list of cols that have both count and perc
cols_to_impute = ['cases_assigned', 'cases_interviewed', 'cases_reached',
                  'contacts_infected', 'contacts_interviewed', 'contacts_named',
                 'contacts_reached']

imputation_dict = {
    #'cases_assigned_count': '?', 
    #'cases_assigned_perc': 'df["cases_assigned_count"]/(df["cases_assigned_count"]+df["cases_not_assigned_count"])*100', 
    #'cases_interviewed_count': '?', 
    'cases_interviewed_perc': 'df["cases_interviewed_count"]/df["cases_assigned_count"]*100',
    #'cases_reached_count': '',
    'cases_reached_perc': 'df["cases_reached_count"]/df["cases_assigned_count"]*100',
    
    #'contacts_named_count': '',
    'percent_naming_contacts': 'df["contacts_named_cases_naming"]/df["cases_interviewed_count"]*100',
    #'contacts_infected_count': '', 
    'contacts_infected_perc': 'df["contacts_infected_count"]/df["contacts_named_count"]*100',
    #'contacts_interviewed_count': '', 
    'contacts_interviewed_perc': 'df["contacts_interviewed_count"]/df["contacts_named_count"]*100',
    #'contacts_reached_count': '',
    'contacts_reached_perc': 'df["contacts_reached_count"]/df["contacts_named_count"]*100'
}

In [44]:
# only inputes is the data is missing
for column, equation in imputation_dict.items():
    df[column] = df.apply(
        lambda row: eval(equation, {}, {"df": row}) if pd.isna(row[column]) else row[column], 
        axis=1
    )

cases_interviewed_perc
df["cases_interviewed_count"]/df["cases_assigned_count"]*100
cases_reached_perc
df["cases_reached_count"]/df["cases_assigned_count"]*100
percent_naming_contacts
df["contacts_named_cases_naming"]/df["cases_interviewed_count"]*100
contacts_infected_perc
df["contacts_infected_count"]/df["contacts_named_count"]*100
contacts_interviewed_perc
df["contacts_interviewed_count"]/df["contacts_named_count"]*100
contacts_reached_perc
df["contacts_reached_count"]/df["contacts_named_count"]*100


In [45]:
df.head()

,Unnamed: 0,source,pm_start_date,pm_end_date,pm_location,adoption_rate,cases_assigned_count,cases_interviewed_count,cases_interviewed_perc,cases_named_contacts_count,...,notifications_sent_count_weekly,notifications_sent_max,num_users_count,num_users_max,num_users_upper_bound,percent_naming_contacts,staff_hired_count,staff_hired_per_10kpop,user_cases_notifying,users_with_code
0,0,A Multifaceted Evaluation of a COVID-19 Contac...,04/01/2021,06/30/2020,King County,NaN,NaN,NaN,76.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A Multifaceted Evaluation of a COVID-19 Contac...,07/01/2020,06/30/2021,King County,NaN,52317.0,42900.0,82.000115,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A Multifaceted Evaluation of a COVID-19 Contac...,07/01/2020,09/30/2020,King County,NaN,NaN,NaN,87.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A Patient-Initiated Digital COVID-19 Contact N...,05/18/2020,08/17/2020,United States*,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,A State Health Department and Health Informati...,06/15/2020,08/28/2021,Maryland,NaN,498739.0,352356.0,70.649378,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df.round(2).to_csv('/Users/kaitlinsinger/wa_notify/DEN_future/zotero_sql_querry/simplified_wide_df_with_date_loc_imputed.csv')